### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
## Need this lib for DeBERTA
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 8.9 MB/s 


In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed evaluate
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

### Load data and model

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/SetFit___json/SetFit--qqp-4b14bf13c8e13797/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/320 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

  0%|          | 0/364 [00:00<?, ?ba/s]

  0%|          | 0/391 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
import evaluate
accuracy_metric = evaluate.load("accuracy")
accuracy_metric.compute(references=[0, 1], predictions=[0, 1])

{'accuracy': 1.0}

In [ ]:
model = model.to(device)

In [ ]:
BATCH = 256
WORKERS = 2

val_set = qqp_preprocessed['validation']

val_loader = torch.utils.data.DataLoader(
    val_set, 
    batch_size=BATCH, 
    shuffle=False, 
    collate_fn=transformers.default_data_collator,
    num_workers=WORKERS,
)
len(val_set)

40430

In [ ]:
%%time
all_predictions = torch.tensor([]).to(device)
all_labels = torch.tensor([]).to(device)

for i, batch in enumerate(val_loader):
    # batch = batch.to(device)
    # batc
    with torch.no_grad():
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )
        predictions = torch.argmax(predicted.logits, dim=1).to(device)
        all_predictions = torch.cat([all_predictions, predictions]).to(device)
        all_labels = torch.cat([all_labels, batch['labels'].to(device)]).to(device)
    if i % 10 == 0 and i != 0:
        print(f'{i} iter')

results = accuracy_metric.compute(references=all_labels, predictions=all_predictions)
print(results)

10 iter
20 iter
30 iter
40 iter
50 iter
60 iter
70 iter
80 iter
90 iter
100 iter
110 iter
120 iter
130 iter
140 iter
150 iter
{'accuracy': 0.9083848627256987}
CPU times: user 4min 18s, sys: 489 ms, total: 4min 19s
Wall time: 4min 24s


In [ ]:
accuracy = results['accuracy']
print('acc =', accuracy)
assert 0.9 < accuracy < 0.91

acc = 0.9083848627256987


### Task 2: train the model (5 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')

---

In [6]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = "funnel-transformer/small"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name);
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2);

In [ ]:
MAX_LENGTH = 128

def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [ ]:
BATCH = 64
WORKERS = 1
EPOCHS = 1

training_args = TrainingArguments(  output_dir='sample_data',
                                    dataloader_num_workers=WORKERS,
                                    per_device_train_batch_size=BATCH,
                                    num_train_epochs=EPOCHS,
                                )

In [ ]:
torch.cuda.empty_cache()
import gc
# del variables
gc.collect()

9088

In [ ]:
import evaluate
import numpy as np
accuracy_metric = evaluate.load("accuracy")
def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

torch.cuda.empty_cache()

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=qqp_preprocessed['train'],
        eval_dataset=qqp_preprocessed['validation'],
        compute_metrics=compute_accuracy,
        tokenizer=tokenizer,
        data_collator=transformers.default_data_collator
    )
# Training
train_result = trainer.train()

The following columns in the training set don't have a corresponding argument in `FunnelForSequenceClassification.forward` and have been ignored: text2, text1, idx, label_text. If text2, text1, idx, label_text are not expected by `FunnelForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 363846
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 5686
  Number of trainable parameters = 116203778


Step,Training Loss
500,0.461200
1000,0.377500
1500,0.345600
2000,0.330700
2500,0.315300
3000,0.299300
3500,0.285000
4000,0.283400
4500,0.271500
5000,0.263900


Saving model checkpoint to sample_data/checkpoint-500
Configuration saved in sample_data/checkpoint-500/config.json
Model weights saved in sample_data/checkpoint-500/pytorch_model.bin
tokenizer config file saved in sample_data/checkpoint-500/tokenizer_config.json
Special tokens file saved in sample_data/checkpoint-500/special_tokens_map.json
Saving model checkpoint to sample_data/checkpoint-1000
Configuration saved in sample_data/checkpoint-1000/config.json
Model weights saved in sample_data/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in sample_data/checkpoint-1000/tokenizer_config.json
Special tokens file saved in sample_data/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to sample_data/checkpoint-1500
Configuration saved in sample_data/checkpoint-1500/config.json
Model weights saved in sample_data/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in sample_data/checkpoint-1500/tokenizer_config.json
Special tokens file saved in sample_dat

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainer.save_model(f"/content/drive/MyDrive/nlp_course/week_5/trained_qqp_{model_name.replace('/', '_')}")

Saving model checkpoint to /content/drive/MyDrive/nlp_course/week_5/trained_qqp_funnel-transformer_small
Configuration saved in /content/drive/MyDrive/nlp_course/week_5/trained_qqp_funnel-transformer_small/config.json
Model weights saved in /content/drive/MyDrive/nlp_course/week_5/trained_qqp_funnel-transformer_small/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/nlp_course/week_5/trained_qqp_funnel-transformer_small/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/nlp_course/week_5/trained_qqp_funnel-transformer_small/special_tokens_map.json


---

### Load model and evaluate accuracy

In [6]:
import evaluate
import numpy as np
accuracy_metric = evaluate.load("accuracy")

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
## load model, tokenizer
## dataset preprocessing
model_name = "funnel-transformer/small"
model_path = f"/content/drive/MyDrive/nlp_course/week_5/trained_qqp_{model_name.replace('/', '_')}"
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_path, )
model = model.to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

  0%|          | 0/364 [00:00<?, ?ba/s]

  0%|          | 0/391 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

In [25]:
BATCH = 1024
WORKERS = 2
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, 
    batch_size=BATCH, 
    shuffle=False, 
    collate_fn=transformers.default_data_collator,
    num_workers=WORKERS,
)

In [24]:
len(val_set)

40430

In [26]:
%%time
all_predictions = torch.tensor([]).to(device)
all_labels = torch.tensor([]).to(device)

for i, batch in enumerate(val_loader):
    # batch = batch.to(device)
    # batc
    with torch.no_grad():
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )
        predictions = torch.argmax(predicted.logits, dim=1).to(device)
        all_predictions = torch.cat([all_predictions, predictions]).to(device)
        all_labels = torch.cat([all_labels, batch['labels'].to(device)]).to(device)
    if i % 10 == 0 and i != 0:
        print(f'{i} iter')


results = accuracy_metric.compute(references=all_labels, predictions=all_predictions)
print(results)

10 iter
20 iter
30 iter
{'accuracy': 0.8915656690576305}
CPU times: user 3min 22s, sys: 301 ms, total: 3min 22s
Wall time: 3min 24s


*I guess, my model has comparable accuracy with the basic model we have used*

### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [ ]:
import evaluate
import numpy as np
accuracy_metric = evaluate.load("accuracy")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## load model, tokenizer
## dataset preprocessing
model_name = "funnel-transformer/small"
model_path = f"/content/drive/MyDrive/nlp_course/week_5/trained_qqp_{model_name.replace('/', '_')}"
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_path, 
                                                                        output_hidden_states=True,
                                                                        num_labels=2)
model = model.to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [54]:
text = """Replace me by any text you'd like. sure of course I can and do this, let's add some Ktulhu stuff here
Replace me by any text you'd like. sure of course I can and do this, let's add some Ktulhu stuff here x2"""
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(len(text), 'characters')
print(len(text.split()), 'words')
for i in range(len(output.hidden_states)):
    print(f'{i} state: {output.hidden_states[i].shape}')

In [ ]:
# def semantic_preprocess(examples, text_num):
#     quest_num = 'text' + str(text_num)
#     another_quest = 'text2' if text_num == 1 else 'text1'
#     result = tokenizer(
#         examples[quest_num], examples[quest_num],
#         padding='max_length', max_length=MAX_LENGTH, truncation=True
#     )
#     result['label'] = [1] * len(examples['label'])
#     result[another_quest] = examples[quest_num]
#     return result

# semantic_text1 = qqp['train'].map(lambda examples: semantic_preprocess(examples, text_num=1), batched=True)
# semantic_text2 = qqp['train'].map(lambda examples: semantic_preprocess(examples, text_num=2), batched=True)

In [ ]:
# from datasets import concatenate_datasets
# semantic_txt = concatenate_datasets([semantic_text1, semantic_text2])
# semantic_txt

In [ ]:
# def get_vectors(model, texts):
#     encoded_input = tokenizer(text, return_tensors='pt')
#     output = model(encoded_input)
#     cls_idx = 0
    
#     #took CLS token representation from the last hidden state
#     vector = output.hidden_states[-1][:][0]

In [ ]:
# %%time

# BATCH = 256
# WORKERS = 2

# val_set = qqp_preprocessed['validation']

# val_loader = torch.utils.data.DataLoader(
#     val_set, 
#     batch_size=BATCH, 
#     shuffle=False, 
#     collate_fn=transformers.default_data_collator,
#     num_workers=WORKERS,
# )

# all_predictions = torch.tensor([]).to(device)
# all_labels = torch.tensor([]).to(device)

# for i, batch in enumerate(val_loader):
#     # batch = batch.to(device)
#     # batc
#     with torch.no_grad():
#         predicted = model(
#             input_ids=batch['input_ids'].to(device),
#             attention_mask=batch['attention_mask'].to(device),
#             token_type_ids=batch['token_type_ids'].to(device)
#         )
#         predictions = torch.argmax(predicted.logits, dim=1).to(device)
#         all_predictions = torch.cat([all_predictions, predictions]).to(device)
#         all_labels = torch.cat([all_labels, batch['labels'].to(device)]).to(device)
#     if i % 10 == 0 and i != 0:
#         print(f'{i} iter')

# results = accuracy_metric.compute(references=all_labels, predictions=all_predictions)
# print(results)

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.